<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasci 266 final project - baseline summarization notebook

**Description:**

**Structure of the notebook:**

**Expected results:**





## Setup

In [1]:
!pip install -q sentencepiece

In [2]:
!pip install -q transformers

In [3]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you 

In [4]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [6]:
# Install Necessary Libraries (if not already installed)
!pip install datasets pandas


In [7]:
# Load the Dataset
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("xsum")

# Convert to Pandas DataFrames for Easier Manipulation
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [8]:
# Explore the Dataset
print("Number of Training Examples:", len(train_df))
print("Number of Validation Examples:", len(validation_df))
print("Number of Test Examples:", len(test_df))
print(train_df.head().to_markdown(index=False, numalign="left", stralign="left")) # Show first 5 rows of the training set in a markdown table


Number of Training Examples: 204045
Number of Validation Examples: 11332
Number of Test Examples: 11334
| document                                                                                                                                                                                                                                                                            | summary                                                                                                                                       | id       |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|:---------|
| The full cost of damage 

## 1. T5 for Generic Summarization

T5 is an encoder decoder architecture that has been trained on multiple tasks, so not purely summarization.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/t5).

In [43]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
t5model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import Dataset

class SummaryEvaluator:
    """
    A class for evaluating text summarization models using ROUGE and BLEU metrics.
    """

    def __init__(self, rouge_metrics=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True):
        """
        Initializes the RougeBleuEvaluator.

        Args:
            rouge_metrics (list): List of ROUGE metrics to calculate (e.g., ['rouge1', 'rouge2', 'rougeL']).
            use_stemmer (bool): Whether to use stemming for calculating ROUGE scores.
        """
        self.rouge_scorer = rouge_scorer.RougeScorer(rouge_metrics, use_stemmer=use_stemmer)
        self.smoothing_function = SmoothingFunction().method4  # Choose a smoothing method
        self.rouge_metrics = rouge_metrics

    def calculate_rouge(self, reference_summaries, generated_summaries):
        """
        Calculates ROUGE scores.
        """
        if isinstance(reference_summaries, Dataset):
            reference_summaries = reference_summaries["summary"]

        rouge_scores = []
        for ref_summary, gen_summary in zip(reference_summaries, generated_summaries):
            scores = self.rouge_scorer.score(ref_summary, gen_summary)
            rouge_scores.append(scores)

        avg_rouge_scores = {
            metric: sum(score[metric].fmeasure for score in rouge_scores) / len(rouge_scores)
            for metric in self.rouge_metrics
        }

        return avg_rouge_scores

    def calculate_bleu(self, reference_summaries, generated_summaries):
        """
        Calculates BLEU scores.
        """

        if isinstance(reference_summaries, Dataset):
            reference_summaries = reference_summaries["summary"]

        bleu_scores = []
        for ref_summary, gen_summary in zip(reference_summaries, generated_summaries):
            # Tokenize summaries into words or subwords (depends on your model)
            reference_tokens = ref_summary.split()
            generated_tokens = gen_summary.split()
            score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=self.smoothing_function)
            bleu_scores.append(score)

        avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

        return avg_bleu_score

    def evaluate(self, reference_summaries, generated_summaries):
        """
        Calculates and prints both ROUGE and BLEU scores.
        """
        avg_rouge_scores = self.calculate_rouge(reference_summaries, generated_summaries)
        avg_bleu_score = self.calculate_bleu(reference_summaries, generated_summaries)

        print("Average ROUGE scores:", avg_rouge_scores)
        print("Average BLEU score:", avg_bleu_score)

        return avg_rouge_scores, avg_bleu_score

In [14]:
from datasets import load_dataset
def load_model_dataset (dataset_name="xsum"):
  return load_dataset(dataset_name)


In [31]:
def load_sampled_dataset (dataset=load_model_dataset(), sample_size=1, seed=42):
  return dataset['train'].shuffle(seed=seed).select(range(sample_size))

In [44]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

def model_summarization(
    sampled_dataset=load_sampled_dataset(),
    seed=42,
    model=t5model,
    tokenizer=t5tokenizer,
    max_length=150,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
    ):
    """
    Evaluates the summarization model on a sample of a dataset using ROUGE metrics.

    Args:
        dataset_name (str): Name of the dataset to load (default is "xsum").
        sample_size (int): Number of examples to sample (default is 1).
        t5model (str): Name of the T5 model to use (default is "t5-small").

    Returns:
        dict: Average ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    # Load T5 tokenizer and model
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

    # Summarize and calculate ROUGE scores (similar to your original code)
    generated_summaries = []
    for example in sampled_dataset:
        summary = summarizer(example['document'],
                             max_length=max_length,
                             min_length=min_length,
                             length_penalty=length_penalty,
                             num_beams=num_beams,
                             early_stopping=early_stopping)[0]['summary_text']
        generated_summaries.append(summary)

    return generated_summaries





In [46]:
# Test on T5 model
sampled_dataset = load_sampled_dataset()
evaluator = SummaryEvaluator()
generated_summaries = model_summarization(sampled_dataset=sampled_dataset)
avg_rouge_scores, avg_bleu_score = evaluator.evaluate(sampled_dataset, generated_summaries)

Average ROUGE scores: {'rouge1': 0.1492537313432836, 'rouge2': 0.0, 'rougeL': 0.11940298507462685}
Average BLEU score: 0.007337132036182038


In [47]:
print(generated_summaries)
print(sampled_dataset['document'][0])
print(sampled_dataset['summary'][0])

["there are already more than 2,000 secondary academies in english schools . governing bodies are responsible for setting school hours and need ministerial permission . academy plans for english schools are not tied to pay scales . there is growing momentum for devolution of teachers' pay and conditions ."]
In Wales, councils are responsible for funding and overseeing schools.
But in England, Mr Osborne's plan will mean local authorities will cease to have a role in providing education.
Academies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.
It is a significant development in the continued divergence of schools systems on either side of Offa's Dyke.
And although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.
Ministers have no plans to follow suit.
At the moment, governing bodies are responsible fo

Let's just run T5 using it's default hyperparameters and see what happens.  We'll hold on to the output in the candidate variable.  What do you think about the output?

In [48]:
#In order to not consume all of the memory available in Colab we'll free up the memory we're using for these large language models
del t5model
del t5tokenizer


## 2. Pegasus for Headline Summarization

Pegasus is an encoder decoder architecture that has been explicitly pre-trained as an abstractive summarizer.  You can read more about it [here](https://huggingface.co/docs/transformers/model_doc/pegasus) and [here](https://arxiv.org/pdf/1912.08777.pdf).

We'll first use the `google/pegasus-xsum` checkpoint.  It is trained on a [summarization task](https://aclanthology.org/D18-1206.pdf) that reads a news article and then [emits a one line summary](https://huggingface.co/datasets/xsum).  This doesn't mean that it is limited in its output length.  It does mean that it works well with news article type inputs and tends toward shorter outputs.

In [49]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [14]:
pmodel.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
 final_logits_bias (BiasLay  multiple                  96103     
 er)                                                             
                                                                 
Total params: 569844583 (2.12 GB)
Trainable params: 569748480 (2.12 GB)
Non-trainable params: 96103 (375.40 KB)
_________________________________________________________________


In [50]:
evaluator = SummaryEvaluator()
generated_summaries = model_summarization(sampled_dataset=sampled_dataset,model=pmodel,tokenizer=ptokenizer)
avg_rouge_scores, avg_bleu_score = evaluator.evaluate(sampled_dataset, generated_summaries)

Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors


Average ROUGE scores: {'rouge1': 0.2692307692307692, 'rouge2': 0.12, 'rougeL': 0.2692307692307692}
Average BLEU score: 0.030623029807752625
